# Erstellen einer Observation Ressource zur Angabe des Blutdrucks

In [ ]:
!pip install git+git://github.com/smart-on-fhir/client-py.git

In [ ]:
from IPython.display import IFrame
import json
from fhirclient.models import (
    observation,
    meta,
    codeableconcept,
    coding,
    fhirreference,
    fhirdate,
    quantity
)

In [ ]:
# Blutdruck Profil in der FHIR Spezifikation.
IFrame('https://hl7.org/fhir/bp.html', width=1200, height=330)

In [ ]:
# Instanziieren der Observation
my_observation = observation.Observation()

In [ ]:
# Hinzufügen von Metadaten über die Ressource
bp_meta = meta.Meta()
bp_meta.profile = ['http://hl7.org/fhir/StructureDefinition/bp']
my_observation.meta = bp_meta

In [ ]:
# Status der Blutdruckmessung
status = 'final'
my_observation.status = status

In [ ]:
# Kategorie der Observation
category = codeableconcept.CodeableConcept()
category_coding = coding.Coding()
category_coding.system = 'http://terminology.hl7.org/CodeSystem/observation-category'
category_coding.code = 'vital-signs'
category.coding = [category_coding]
my_observation.category = [category]

In [ ]:
# Der LOINC Code für diese Observation
bp_code = codeableconcept.CodeableConcept()
code_coding = coding.Coding()
code_coding.system = 'http://loinc.org'
code_coding.code = '85354-9'
code_coding.display = 'Blood pressure panel with all children optional'
bp_code.coding = [code_coding]
my_observation.code = bp_code

In [ ]:
# Patienten aus Patient-leicht Übung holen
import requests

fhir_test_server = 'https://vonk.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

response = requests.get(url=f'{fhir_test_server}/Patient?family=Holmes', headers=headers)

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [ ]:
# Response Text als Python Dictionary holen
response_dict = json.loads(response.text)
response_dict

In [ ]:
# Referenz zum Patienten
subject = fhirreference.FHIRReference()
pat_id = response_dict['id']
subject.reference = f'Patient/{pat_id}'
my_observation.subject = subject

In [ ]:
# Zeitpunkt der Messung
date = fhirdate.FHIRDate('2020-12-02')
my_observation.effectiveDateTime = date

In [ ]:
print(json.dumps(my_observation.as_json(), indent=4))

## Aufgabe

Dieser Blutdruckmessung fehlen ja noch die Messwerte. Überprüfen Sie die Vorgaben vom [FHIR Blurdruck-Profil](https://hl7.org/fhir/bp). 

------
**Ihre Aufgabe:** Implementieren Sie die fehlenden Komponenten, sodass systolischer und diastolischer Wert vorhanden sind. (Tipp: schauen Sie sich das Datenelement ObservationComponent der Observation Ressource an)

In [ ]:
# systolischer Blutdruck
systolic_component = observation.ObservationComponent()

systolic_code = codeableconcept.CodeableConcept()
systolic_coding = coding.Coding()
systolic_coding.system = 'http://loinc.org'
systolic_coding.code = '8480-6'
systolic_coding.display = 'Systolic blood pressure'
systolic_code.coding = [systolic_coding]

systolic_component.code = systolic_code

systolic_value = quantity.Quantity()
systolic_value.value = 120
systolic_value.unit = 'mmHg'
systolic_value.system = 'http://unitsofmeasure.org'
systolic_value.code = 'mm[Hg]'

systolic_component.valueQuantity = systolic_value

print(json.dumps(systolic_component.as_json(), indent=4))

In [ ]:
# diastolischer Blutdruck
diastolic_component = observation.ObservationComponent()

diastolic_code = codeableconcept.CodeableConcept()
diastolic_coding = coding.Coding()
diastolic_coding.system = 'http://loinc.org'
diastolic_coding.code = '8462-4'
diastolic_coding.display = 'Diastolic blood pressure'
diastolic_code.coding = [diastolic_coding]

diastolic_component.code = diastolic_code

diastolic_value = quantity.Quantity()
diastolic_value.value = 80
diastolic_value.unit = 'mmHg'
diastolic_value.system = 'http://unitsofmeasure.org'
diastolic_value.code = 'mm[Hg]'

diastolic_component.valueQuantity = diastolic_value

print(json.dumps(diastolic_component.as_json(), indent=4))

In [ ]:
my_observation.component = [systolic_component, diastolic_component]

In [ ]:
print(json.dumps(my_observation.as_json(), indent=4))

In [ ]:
# Validieren gegen den FHIR Server
import requests

In [ ]:
fhir_test_server = 'https://vonk.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

In [ ]:
response = requests.post(f'{fhir_test_server}/Observation/$validate', headers = headers, data = json.dumps(my_observation.as_json()))
data = response.json()

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [ ]:
print(json.dumps(data, indent=4))